In [1]:
!pip install opencv-python
!pip install keras
!pip install tensorflow

  Using cached opencv_python-4.8.1.78-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
Using cached opencv_python-4.8.1.78-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (61.7 MB)
  Using cached keras-3.0.1-py3-none-any.whl.metadata (4.8 kB)
  Using cached absl_py-2.0.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached rich-13.7.0-py3-none-any.whl.metadata (18 kB)
  Using cached namex-0.0.7-py3-none-any.whl (5.8 kB)
  Using cached dm_tree-0.1.8-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (152 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)
Using cached keras-3.0.1-py3-none-any.whl (999 kB)
Using cached absl_py-2.0.0-py3-none-any.whl (130 kB)
Using cached rich-13.7.0-py3-none-any.whl (240 kB)
Using cached markdown_it_py-3.0.0-py3-none-any.whl (87 kB)
  Using cached tensorflow-2.15.0.post1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4

In [2]:
import numpy as np
import cv2

import matplotlib.pyplot as plt
from keras.datasets import cifar10

from tmu.models.classification.vanilla_classifier import TMClassifier
from time import time

2023-12-10 21:24:18.507798: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-10 21:24:18.507847: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-10 21:24:18.509759: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-10 21:24:18.519649: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-10 21:24:19.334935: W tensorflow/comp

In [3]:
device = "GPU"
max_included_literals = 32
resolution = 8
factor = 1

In [4]:
import cv2
from matplotlib import pyplot as plt
from keras.datasets import cifar10

# Load CIFAR-10 dataset
(X_train_org, Y_train), (X_test_org, Y_test) = cifar10.load_data()

Y_train = Y_train.reshape(Y_train.shape[0])
Y_test = Y_test.reshape(Y_test.shape[0])


X_train_thermometer = np.empty((X_train_org.shape[0], X_train_org.shape[1], X_train_org.shape[2], X_train_org.shape[3], resolution), dtype=np.uint8)
for z in range(resolution):
    X_train_thermometer[:, :, :, :, z] = X_train_org[:, :, :, :] >= (z + 1) * 255 / (resolution + 1)

X_test_thermometer = np.empty((X_test_org.shape[0], X_test_org.shape[1], X_test_org.shape[2], X_test_org.shape[3], resolution), dtype=np.uint8)
for z in range(resolution):
    X_test_thermometer[:, :, :, :, z] = X_test_org[:, :, :, :] >= (z + 1) * 255 / (resolution + 1)

X_train_thermometer = X_train_thermometer.reshape((X_train_org.shape[0], X_train_org.shape[1], X_train_org.shape[2], 3 * resolution))
X_test_thermometer = X_test_thermometer.reshape((X_test_org.shape[0], X_test_org.shape[1], X_test_org.shape[2], 3 * resolution))

In [ ]:
from tmu.models.classification.vanilla_classifier import TMClassifier
from time import time

num_clauses=2000 * factor
T=1500 * factor
s=2.5
patch_size=5
max_included_literals=32
weighted_clauses=True

tm = TMClassifier(
    number_of_clauses=num_clauses,
    T=T,
    s=s,
    max_included_literals=max_included_literals,
    platform=device,
    weighted_clauses=True,
    patch_dim=(patch_size, patch_size),
)

epochs = 100

for epoch in range(epochs):
        start_training = time()
        tm.fit(X_train_thermometer, Y_train)
        stop_training = time()

        start_testing = time()
        Y_test_predicted, Y_test_scores = tm.predict(X_test_thermometer, return_class_sums=True)
        stop_testing = time()

        result_test = 100*(Y_test_scores.argmax(axis=1) == Y_test).mean()

        print("#%d Accuracy: %.2f%% Training: %.2fs Testing: %.2fs" % (epoch+1, result_test, stop_training-start_training, stop_testing-start_testing))

        np.savetxt("5x5ColorThermometers_%d_%d_%d_%.1f_%d_%d_%d.txt" % (epoch+1, num_clauses, T, s, patch_size, max_included_literals, weighted_clauses), Y_test_scores, delimiter=',')


2023-12-10 21:24:44,869 - tmu.clause_bank.clause_bank_cuda - INFO - Loading compiled CUDA module from '/tmp/tm_kernels/7e510798429bc0446f124b07a80a84cc862fe285.ptx'.
2023-12-10 21:24:44,872 - tmu.clause_bank.clause_bank_cuda - INFO - Loading compiled CUDA module from '/tmp/tm_kernels/e874c77c0ef8afd7d3b305f1fed9aa0707d3a563.ptx'.
2023-12-10 21:24:44,874 - tmu.clause_bank.clause_bank_cuda - INFO - Loading compiled CUDA module from '/tmp/tm_kernels/d29e3375674f81bbbbf1a534b5960ac4ed35523f.ptx'.
2023-12-10 21:24:44,876 - tmu.clause_bank.clause_bank_cuda - INFO - Loading compiled CUDA module from '/tmp/tm_kernels/9b014f834007e4ac7da33eca46bde05bcb326148.ptx'.
2023-12-10 21:24:45,024 - tmu.clause_bank.clause_bank_cuda - INFO - Loading compiled CUDA module from '/tmp/tm_kernels/7e510798429bc0446f124b07a80a84cc862fe285.ptx'.
2023-12-10 21:24:45,026 - tmu.clause_bank.clause_bank_cuda - INFO - Loading compiled CUDA module from '/tmp/tm_kernels/e874c77c0ef8afd7d3b305f1fed9aa0707d3a563.ptx'.
2023